In [1]:
import pandas as pd 
import json
import re
import numpy as np 
import torch
import tqdm
import pickle
from timeit import default_timer as timer
from math import floor
import random


## Review.json을 text file로

In [2]:
data = []
with open('../Librarything/reviews.json', 'r', encoding='UTF-8') as f:
   for line in f:
        line = json.dumps(line)
        data.append(json.loads(line))

In [3]:
item = pd.DataFrame(data)

In [4]:
pd.set_option('display.max.colwidth', 100)

In [6]:
temp = []
exception = []
for index,value in item.iterrows():
    first = value[0].find("'comment'")
    second = value[0].find("'user'")
    content = value[0][first+12:second-3].replace("\"","'")
    content = content.replace("\\",'')
    review = "\""+content+"\""
    change = value[0].replace("'","\"")
    word = change[:first+11]+review+change[second-2:]
    try:
        temp.append(json.loads(word))
    except:
        exception.append([index, word])


In [7]:
dataframe = pd.DataFrame(temp)
df = dataframe
del df['flags']
del df['stars']

In [9]:
df.head()

,work,unixtime,nhelpful,time,comment,user
0,3206242,1194393600,0,"Nov 7, 2007",This a great book for young readers to be introduced to the world of Middle Earth.,van_stef
1,12198649,1333756800,0,"Apr 7, 2012",Help Wanted: Tales of On The Job Terror from Evil Jester Press is a fun and scary read. This boo...,dwatson2
2,12533765,1352937600,0,"Nov 15, 2012","Magoon, K. (2012). Fire in the streets. New York: Simon and Schuster/Aladdin. 336 pp. ISBN: 978-...",edspicer
3,12981302,1364515200,0,"Mar 29, 2013","Well, I definitely liked this book better than the last in the series. There was less fighting a...",amdrane2
4,5231009,1270944000,0,"Apr 11, 2010",It's a nice science-fiction thriller with some Christian elements. It was a little difficult for...,Lila_Gustavus


In [13]:
print("전처리 이전의 User 수 : ", len(df['user'].unique()))
print("전처리 이전의 item 수 : ", len(df['work'].unique()))

전처리 이전의 User 수 :  83195
전처리 이전의 item 수 :  506165


In [73]:
del_index = []
item_index = {}
user_index = {}
def delete_func(x):
    if len(x) <= 1:
        del_index.extend(x.index)

In [80]:
start =timer()
del_index = []
df.groupby(['user']).apply(delete_func)
print("전처리 시간 : ", timer()-start)
df = df.drop(del_index)
print("Unique User의 수 : ", len(df['user'].unique()))
print("Unique Item의 수 : ", len(df['work'].unique()))

전처리 시간 :  3.453278817003593
Unique User의 수 :  57208
Unique Item의 수 :  506165


In [81]:
start =timer()
del_index = []
df.groupby(['work']).apply(delete_func)
print("전처리 시간 : ", timer()-start)
df = df.drop(del_index)
print("Unique User의 수 : ", len(df['user'].unique()))
print("Unique Item의 수 : ", len(df['work'].unique()))

전처리 시간 :  10.003817755030468
Unique User의 수 :  57208
Unique Item의 수 :  168847


In [82]:
start = timer()
item_index = {}
user_index = {}
user_set = set()
item_set = set()
u_index = 0
i_index = 0
for index, value in df.iterrows():
    if value['user'] not in user_set:
        user_index[value['user']] = u_index
        user_set.add(value['user'])
        u_index += 1
    if value['work'] not in item_set:
        item_index[value['work']] = i_index
        item_set.add(value['work'])
        i_index += 1
print("총 전처리 시간 : ", timer()-start)

총 전처리 시간 :  116.21273380203638


In [83]:
df['user'] = df['user'].map(lambda x: user_index[x])

In [84]:
df['work'] = df['work'].map(lambda x: item_index[x])

In [90]:
inter_dict = {}
def interaction_func(x):
    inter_dict[x.name] = list(x['work'])
df.groupby(by=['user']).apply(interaction_func)

""


In [109]:
with open('../Librarything/total_data.txt', 'w') as output:
    for k, v in inter_dict.items():
        line = [k] + v
        output.write(" ".join(map(str,line)))
        output.write('\n')


## Train과 test 0.8, 0.2로 나눈 뒤 실행

In [110]:
train_set = {}
test_set = {}
with open('../Librarything/total_data.txt', 'r') as output:
    for l in output.readlines():
        l = l.strip('\n').split(' ')
        items = [int(i) for i in l[1:]]
        uid = int(l[0])
        train_length = floor(len(items)*0.8)
        train_set[uid] = random.sample(items, train_length)
        test_set[uid] = list(set(items) - set(train_set[uid]))
    

In [98]:
with open('../Librarything/train.txt', 'w') as output:
    for k, v in train_set.items():
        temp_list = [k]+v
        output.write(" ".join(map(str, temp_list)))
        output.write('\n')
with open('../Librarything/test.txt', 'w') as output:
    for k, v in test_set.items():
        temp_list = [k]+v
        output.write(" ".join(map(str, temp_list)))
        output.write('\n')

## user에게 index만들고 저장

In [105]:
map_user_df = pd.DataFrame(zip(user_index.keys(),user_index.values()), columns= ['org_id', 'remap_id'])

In [106]:
print(map_user_df.head())

          org_id  remap_id
0       van_stef         0
1       dwatson2         1
2       edspicer         2
3       amdrane2         3
4  Lila_Gustavus         4


In [108]:
map_user_df.to_csv('../Librarything/user_list.txt', sep = '\t', index =False)

## Item에 index만들고 저장

In [111]:
map_df = pd.DataFrame(zip(item_index.keys(),item_index.values()), columns= ['org_id', 'remap_id'])

In [113]:
map_df.to_csv('../Librarything/item_list.txt', sep = '\t', index = False)